## Image网 Submission `128x128`

This contains a submission for the Image网 leaderboard in the `128x128` category.

In this notebook we:
1. Train on 1 pretext task: 
  - Train a network to do image inpatining on Image网's `/train`, `/unsup` and `/val` images. 
2. Train on 4 downstream tasks:
  - We load the pretext weights and train for `5` epochs.
  - We load the pretext weights and train for `20` epochs.
  - We load the pretext weights and train for `80` epochs.
  - We load the pretext weights and train for `200` epochs.
  
Our leaderboard submissions are the accuracies we get on each of the downstream tasks.

In [1]:
import json
import torch

import numpy as np

from functools import partial
from fastai2.basics import *
from fastai2.vision.all import *

In [2]:
torch.cuda.set_device(2)

In [3]:
# Chosen parameters
lr=2e-2
sqrmom=0.99
mom=0.95
beta=0.
eps=1e-4
bs=64 
sa=1
wd=1e-3
m = xse_resnext18
act_fn = Mish
pool = MaxPool

nc=20

In [4]:
source = untar_data(URLs.IMAGEWANG_160)
len(get_image_files(source/'unsup')), len(get_image_files(source/'train')), len(get_image_files(source/'val'))

(7750, 14669, 3929)

In [5]:
# Use the Ranger optimizer
opt_func = partial(ranger, mom=mom, sqr_mom=sqrmom, eps=eps, beta=beta)

In [6]:
m_part = partial(m, c_out=nc, act_cls=act_fn, sa=sa, pool=pool)
model_meta[m_part] = model_meta[xresnet18]

In [7]:
save_name = 'imagewang_contrast_kornia_xse_resnext18_doublebs'

In [8]:
#export
from pytorch_metric_learning import losses
class XentLoss(losses.NTXentLoss):
    def forward(self, output1, output2):
        stacked = torch.cat((output1, output2), dim=0)
        labels = torch.arange(output1.shape[0]).repeat(2)
        return super().forward(stacked, labels, None)
    

In [9]:
# bs,feat=3,7 # 
# targ = torch.randn(bs, feat)*5 # bs x features
# aug = targ+0.1 # bs x features
# rand_aug = torch.randn(bs, feat) # bs x features
# temp = 0.5

# closs = XentContrastiveLoss(temp)
# closs(targ,rand_aug)

# closs2 = XentLoss(temp)
# closs2(targ,rand_aug)

In [10]:
class XentContrastiveLoss(nn.Module):
    def __init__(self, temp=0.5):
        super().__init__()
        self.temp = temp

    def forward(self, x, y):
        bs,feat = x.shape
        t2 = x.view(bs,1,feat)
        
        csim = F.cosine_similarity(x, t2, dim=-1)/self.temp
        csim_noself = csim[~torch.eye(bs).bool()].reshape(bs, bs-1)
        return F.cross_entropy(csim_noself, y)
    

In [11]:
# class XentContrastiveLoss(nn.Module):
#     def __init__(self, temp=0.5):
#         super().__init__()
#         self.temp = temp

#     def forward(self, output1, output2):
#         x = torch.cat((output1, output2), dim=0)
#         bs,feat = x.shape
#         y = x.view(bs,1,feat)
        
#         csim = F.cosine_similarity(x, y, dim=-1)/self.temp
#         csim_noself = csim[~torch.eye(bs).bool()].reshape(bs, bs-1)
        
#         labels = (torch.arange(bs)+bs//2-1) % (bs-1)
#         return F.cross_entropy(csim_noself, labels)
    

## Pretext Task: Contrastive Learning

In [12]:
#export
from pytorch_metric_learning import losses
class XentLoss(losses.NTXentLoss):
    def forward(self, output1, output2):
        stacked = torch.cat((output1, output2), dim=0)
        labels = torch.arange(output1.shape[0]).repeat(2)
        return super().forward(stacked, labels, None)
    
class ContrastCallback(Callback):
    run_before=Recorder
    def __init__(self, size=256, aug_targ=None, aug_pos=None, temperature=0.1):
        self.aug_targ = ifnone(aug_targ, get_aug_pipe(size))
        self.aug_pos = ifnone(aug_pos, get_aug_pipe(size))
        self.temperature = temperature
        
    def update_size(self, size):
        pipe_update_size(self.aug_targ, size)
        pipe_update_size(self.aug_pos, size)
        
    def begin_fit(self): 
        self.old_lf = self.learn.loss_func
        self.old_met = self.learn.metrics
        self.learn.metrics = []
#         self.learn.loss_func = XentLoss(self.temperature)
        self.learn.loss_func = XentContrastiveLoss(self.temperature)
        
    def after_fit(self):
        self.learn.loss_fun = self.old_lf
        self.learn.metrics = self.old_met
        
    def begin_batch(self):
        xb, = self.learn.xb
        xb_targ = self.aug_targ(xb)
        xb_pos = self.aug_pos(xb)
        self.learn.xb = torch.cat((xb_targ, xb_pos), dim=0),
        bs = self.learn.xb[0].shape[0]
        self.learn.yb = (torch.arange(bs, device=xb.device)+bs//2-1) % (bs-1),
        

In [13]:
#export
def pipe_update_size(pipe, size):
    for tf in pipe.fs:
        if isinstance(tf, RandomResizedCropGPU):
            tf.size = size

In [14]:
def get_dbunch(size, bs, workers=8, dogs_only=False):
    path = URLs.IMAGEWANG_160 if size <= 160 else URLs.IMAGEWANG
    source = untar_data(path)
    
    folders = ['unsup', 'val'] if dogs_only else None
    files = get_image_files(source, folders=folders)
    
    tfms = [[PILImage.create, ToTensor, RandomResizedCrop(size, min_scale=0.9)], 
            [parent_label, Categorize()]]
    
#     dsets = Datasets(files, tfms=tfms, splits=GrandparentSplitter(train_name='unsup', valid_name='val')(files))
    dsets = Datasets(files, tfms=tfms, splits=RandomSplitter(valid_pct=0.1)(files))
    
#     batch_tfms = [IntToFloatTensor, *aug_transforms(p_lighting=1.0, max_lighting=0.9)]
    batch_tfms = [IntToFloatTensor]
    dls = dsets.dataloaders(bs=bs, num_workers=workers, after_batch=batch_tfms)
    dls.path = source
    return dls

In [15]:
size = 128
bs = 512

dbunch = get_dbunch(160, bs)
len(dbunch.train.dataset)

23714

In [16]:
# dbunch.show_batch()

In [17]:
# # xb = TensorImage(torch.randn(1, 3,128,128))
# afn_tfm, lght_tfm = aug_transforms(p_lighting=1.0, max_lighting=0.8, p_affine=1.0)
# # lght_tfm.split_idx = None
# xb.allclose(afn_tfm(xb)), xb.allclose(lght_tfm(xb, split_idx=0))

In [18]:
import kornia

In [20]:
#export
def get_aug_pipe(size, stats=None, s=.7, scale=(0.16, 1.0)):
    stats = ifnone(stats, imagenet_stats)
    rrc = kornia.augmentation.RandomResizedCrop((size,size), scale=scale, ratio=(3/4, 4/3))
    rhf = kornia.augmentation.RandomHorizontalFlip()
    rcj = kornia.augmentation.ColorJitter(0.8*s, 0.8*s, 0.8*s, 0.2*s)
    
    tfms = [rrc, rhf, rcj, Normalize.from_stats(*stats)]
    pipe = Pipeline(tfms)
    pipe.split_idx = 0
    return pipe

In [21]:
aug = get_aug_pipe(size)
aug2 = get_aug_pipe(size)
cbs = ContrastCallback(size=size, aug_targ=aug, aug_pos=aug2, temperature=0.1)

In [22]:
# xb,yb = dbunch.one_batch()
# nrm = Normalize.from_stats(*imagenet_stats)
# xb_dec = nrm.decodes(aug(xb))
# show_images([xb_dec[0], xb[0]])

In [23]:
ch = nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(512, 256), nn.ReLU(), nn.Linear(256, 128))
learn = cnn_learner(dbunch, m_part, opt_func=opt_func,
                    metrics=[], loss_func=CrossEntropyLossFlat(), cbs=cbs, pretrained=False,
                    config={'custom_head':ch}
                   )
learn.to_fp16();

In [24]:

# state_dict = torch.load(f'imagewang_contrast_kornia.pth')
# learn.model[0].load_state_dict(state_dict, strict=False)

In [25]:
learn.unfreeze()
learn.fit_one_cycle(100, 1e-1, wd=1e-2)

epoch,train_loss,valid_loss,time
0,6.677350,6.205276,01:40
1,6.205480,6.027835,01:30
2,5.792511,5.572533,01:30
3,5.529205,5.246805,01:30
4,5.196542,5.053041,01:30
5,4.860600,4.818596,01:30
6,4.603790,4.844907,01:30
7,4.508495,4.222852,01:31
8,4.357094,3.727122,01:30
9,4.370498,4.546971,01:31


In [26]:
torch.save(learn.model[0].state_dict(), f'{save_name}.pth')

In [27]:
# learn.save(save_name)

## Downstream Task: Image Classification

In [28]:
def get_dbunch(size, bs, workers=8, dogs_only=False):
    path = URLs.IMAGEWANG_160 if size <= 160 else URLs.IMAGEWANG
    source = untar_data(path)
    
    if dogs_only:
        dog_categories = [f.name for f in (source/'val').ls()]
        dog_train = get_image_files(source/'train', folders=dog_categories)
        valid = get_image_files(source/'val')
        files = dog_train + valid
        splits = [range(len(dog_train)), range(len(dog_train), len(dog_train)+len(valid))]
    else:
        files = get_image_files(source)
        splits = GrandparentSplitter(valid_name='val')(files)
        
    
    item_aug = [RandomResizedCrop(size, min_scale=0.35), FlipItem(0.5)]
    tfms = [[PILImage.create, ToTensor, *item_aug], 
            [parent_label, Categorize()]]
    
    dsets = Datasets(files, tfms=tfms, splits=splits)
    
    batch_tfms = [IntToFloatTensor, Normalize.from_stats(*imagenet_stats)]
#     batch_tfms = [IntToFloatTensor, Normalize.from_stats(*imagenet_stats), *aug_transforms()]
    dls = dsets.dataloaders(bs=bs, num_workers=workers, after_batch=batch_tfms)
    dls.path = source
    return dls

In [29]:
def do_train(size=128, bs=64, lr=1e-2, epochs=5, runs=5, dogs_only=False, save_name=None):
    dbunch = get_dbunch(size, bs, dogs_only=dogs_only)
    for run in range(runs):
        print(f'Run: {run}')
        ch = nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(512, 20))
        learn = cnn_learner(dbunch, m_part, opt_func=opt_func, normalize=False,
                metrics=[accuracy,top_k_accuracy], loss_func=LabelSmoothingCrossEntropy(),
                pretrained=False,
                config={'custom_head':ch})

        if save_name is not None:
            state_dict = torch.load(f'{save_name}.pth')
            learn.model[0].load_state_dict(state_dict)

#             state_dict = torch.load('imagewang_inpainting_15_epochs_nopretrain.pth')
#             learn.model[0].load_state_dict(state_dict)
        learn.unfreeze()
        learn.fit_flat_cos(epochs, lr, wd=1e-2)

### 5 Epochs

In [30]:
epochs = 5
runs = 1

In [31]:
do_train(epochs=epochs, runs=runs, lr=2e-2, dogs_only=False, save_name=save_name)

Run: 0


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.188769,2.027163,0.469840,0.852634,00:25
1,1.068448,2.044930,0.470349,0.851616,00:22
2,0.991236,2.007324,0.478239,0.871723,00:22
3,0.941753,2.022613,0.501654,0.862306,00:22
4,0.841646,1.783778,0.565793,0.888521,00:22


### 20 Epochs

In [32]:
epochs = 20
runs = 1

In [33]:
# not inpainting model
do_train(epochs=epochs, runs=runs, lr=1e-2, dogs_only=False, save_name=save_name)

Run: 0


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.200299,2.150598,0.421481,0.817256,00:22
1,1.069303,1.908145,0.499109,0.884703,00:23
2,1.020872,2.001460,0.494782,0.858997,00:22
3,0.948876,1.957461,0.497582,0.865615,00:23
4,0.919015,1.870993,0.540596,0.868923,00:22
5,0.869982,1.893325,0.531433,0.877832,00:22
6,0.846650,1.950099,0.536269,0.869432,00:23
7,0.813176,1.884222,0.556376,0.880886,00:23
8,0.787838,1.957628,0.534742,0.864342,00:23
9,0.770795,1.970407,0.544413,0.861542,00:23


## 80 epochs

In [34]:
epochs = 80
runs = 1

In [35]:
do_train(epochs=epochs, runs=runs, dogs_only=False, save_name=save_name)

Run: 0


KeyboardInterrupt: 

In [ ]:
do_train(epochs=epochs, runs=runs, dogs_only=False, save_name=None)

Accuracy: **62.18%**

### 200 epochs

In [ ]:
epochs = 200
runs = 1

In [ ]:
do_train(epochs=epochs, runs=runs, dogs_only=False, save_name=save_name)

Accuracy: **62.03%**